### Conexiones y parámetros:

In [177]:
import pandas as pd
import pyodbc 
from datetime import date,datetime, timedelta
from dateutil.relativedelta import relativedelta
from pandas.tseries.offsets import MonthEnd
import pandas as pd

pd.options.display.float_format = '{:.2f}'.format

def leer_consulta(ruta_archivo):
    with open(ruta_archivo,'r',encoding='latin-1') as archivo:
        return archivo.read()


print('Librerias Cargadas.')
server = 'ECBPPRQ121,11124' 
database = 'DATALAB_IDN' 
username = 'mencarna' 
password = 'Pelus@141516' 
stringConn ='DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password+';Trusted_Connection=True;';
cnxn = pyodbc.connect(stringConn)
cursor = cnxn.cursor()
print('Conección Exitosa.')
print('Parámetros')
# Parámetros del proceso
Anio = 2025
Mes = 11
Indicador = 'Cierre'  # Opciones: 'Avance', 'Cierre'
ajuste = 'Si'  # Opciones: 'Si', 'No'
Act_usuarios = 'No'
# Mensajes de parámetros
print(f"Se correrá el proceso para: {Indicador}.")
print(f"¿Se realizará un ajuste? {ajuste}.")
# Definición de fechas y listas según el indicador
if Indicador == 'Cierre':
   lista = [
       '[DATALAB_MISMEDIOSPAGO].dbo.ZP_BP_Idn_TS_ProductividadTarjetas',
       '[DATALAB_IDN].[IDN].[SeguimientoAgenciasCierrePasivos]',
       '[DATALAB_IDN].[IDN].[SeguimientoAgenciasCierreCarteraPorVencer]',
       '[DATALAB_IDN].[IDN].[SeguimientoAgenciasCierreCarteraVencida]',
       '[DATALAB_IDN].[IDN].[SeguimientoAgenciasCierrePasivosInternacional]',
       '[DATALAB_IDN].[IDN].[SeguimientoAgenciasCierreCarteraInternacional]',
       '[DATALAB_MISMEDIOSPAGO].dbo.ZP_BP_Idn_TS_ProductividadMasivoCartera'
   ]
   fecha_corte = datetime(Anio, Mes, 1).date() + relativedelta(months=1) - relativedelta(days=1)
else:
   lista = [
       '[DATALAB_MISMEDIOSPAGO].dbo.ZP_BP_Idn_TS_ProductividadTarjetas',
       '[BDDELIVERY].[IDN].[BaseReportePasivo]',
       '[DATALAB_IDN].[HIS].[WORK_RCP_CarteraVencida]',
       '[DATALAB_IDN].[HIS].[WORK_RCP_CarteraVencida]',
       '[DATALAB_IDN].[UIO\erisimba].[VEHICULOS_AUMS_MIAMI_PANAMA]',
       '[DATALAB_IDN].[UIO\erisimba].[VEHICULOS_AUMS_MIAMI_PANAMA]',
       '[DATALAB_MISMEDIOSPAGO].dbo.ZP_BP_Idn_TS_ProductividadMasivoCartera'
   ]
   fecha_corte = date.today()
# Mensajes de variables del proceso
print(f"Variables del proceso: Año: {Anio}, Mes: {Mes}, Fecha de corte: {fecha_corte}.")
print(f"Se correrá el proceso para Año: {Anio} y Mes: {Mes}.")

<>:52: SyntaxWarning: invalid escape sequence '\e'
<>:53: SyntaxWarning: invalid escape sequence '\e'
<>:52: SyntaxWarning: invalid escape sequence '\e'
<>:53: SyntaxWarning: invalid escape sequence '\e'
C:\Users\mencarna\AppData\Local\Temp\ipykernel_23280\3893796271.py:52: SyntaxWarning: invalid escape sequence '\e'
  '[DATALAB_IDN].[UIO\erisimba].[VEHICULOS_AUMS_MIAMI_PANAMA]',
C:\Users\mencarna\AppData\Local\Temp\ipykernel_23280\3893796271.py:53: SyntaxWarning: invalid escape sequence '\e'
  '[DATALAB_IDN].[UIO\erisimba].[VEHICULOS_AUMS_MIAMI_PANAMA]',


Librerias Cargadas.
Conección Exitosa.
Parámetros
Se correrá el proceso para: Avance.
¿Se realizará un ajuste? Si.
Variables del proceso: Año: 2025, Mes: 12, Fecha de corte: 2025-12-09.
Se correrá el proceso para Año: 2025 y Mes: 12.


### Reales

In [178]:
import os
# Ruta de los archivos SQL
ruta_fstp = '//ECUIO197H02/group_files/Inteligencia de Negocios/IDN-Global/Agencias/QuerysAgencias/'
# Listar todos los archivos .sql en la ruta
archivos_sql = [archivo for archivo in os.listdir(ruta_fstp) if archivo.endswith('.sql')]
# Leer y concatenar las consultas usando "UNION ALL"
consulta_final = '\nUNION ALL\n'.join([f'{leer_consulta(ruta_fstp + archivo)}' for archivo in archivos_sql])
consulta_final = consulta_final.format(lista = lista, Anio = Anio, Mes = Mes)
df_Real = pd.read_sql(consulta_final, cnxn)
print('La obtención de Datos Reales de la Fuerza de Venta es Exitosa.')

C:\Users\mencarna\AppData\Local\Temp\ipykernel_23280\1595577542.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_Real = pd.read_sql(consulta_final, cnxn)


La obtención de Datos Reales de la Fuerza de Venta es Exitosa.


## Estructura de Metas

In [179]:
import pandas as pd
ruta_fstp = '//ECUIO197H02/group_files/Inteligencia de Negocios/IDN-Global/Agencias/'
ruta = ruta_fstp+'MetasAgencias'+str(Anio)+str(Mes)+'.xlsx'
df_Metas = pd.read_excel(ruta,
                   header=[0,1,2],
                   index_col=[0,1],
                   sheet_name='Metas')
#df_Metas.drop(columns = "TIPO AGENCIA", inplace = True)
df_Metas.fillna(0, inplace = True)
df_Metas = df_Metas.stack(level=1).stack(level=1).stack().unstack(level=2)
df_Metas = df_Metas.reset_index()
df_Metas=df_Metas.rename(columns = {'level_0':'CodigoAgenciaAnclaje', 'level_1': 'AGENCIA' ,'AGENCIA':'Producto','level_3':'CATEGORIA', 
                                   'METAS': 'Metas'})
df_Metas = df_Metas[['Producto','CodigoAgenciaAnclaje','Metas']]
df_Metas['Producto'] = df_Metas['Producto'].str.strip()

C:\Users\mencarna\AppData\Local\Temp\ipykernel_23280\3497811093.py:10: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  df_Metas = df_Metas.stack(level=1).stack(level=1).stack().unstack(level=2)
C:\Users\mencarna\AppData\Local\Temp\ipykernel_23280\3497811093.py:10: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  df_Metas = df_Metas.stack(level=1).stack(level=1).stack().unstack(level=2)


In [180]:
df_Metas_Tipo_Agencia = pd.read_excel(ruta,
                       header=[0],
                       sheet_name='Agencia')

df_Metas_Tipo_Agencia = df_Metas_Tipo_Agencia[['COD_AGENCIA', 'TIPO']]
df_Metas_Tipo_Agencia = df_Metas_Tipo_Agencia.rename(columns = {'COD_AGENCIA':'CodigoAgenciaAnclaje'})

df_Metas = df_Metas.merge(df_Metas_Tipo_Agencia, how = 'left',
                           left_on = ['CodigoAgenciaAnclaje'], 
                            right_on = [ 'CodigoAgenciaAnclaje']
                           )
if df_Metas_Tipo_Agencia.shape[0]*len(df_Metas['Producto'].unique()) == df_Metas.shape[0]:
    print("Proceso correcto")
else:
    print("Revisar: Agencias Duplicados.")

Proceso correcto


## Lógica reglas

In [181]:
import numpy as np

df_logica = pd.read_excel(ruta,
                   sheet_name='Logica')
df_logica['PRODUCTO'] = np.where(
                                df_logica['PRODUCTO'] == 'APP BANCA MÓVIL', 'APP MÓVIL',
                                df_logica['PRODUCTO'] 
)
df_logica_agencia = df_logica[ (df_logica['TIPO'] == 'AGENCIA TIPO 1') | 
                               (df_logica['TIPO'] == 'AGENCIA TIPO 2') | 
                               (df_logica['TIPO'] == 'AGENCIA TIPO 3') | 
                               (df_logica['TIPO'] == 'AGENCIA ESPECIALIZADA')|
                               (df_logica['TIPO'] == 'AGENCIA ESPECIALIZADA 1')| 
                               (df_logica['TIPO'] == 'AGENCIA ESPECIALIZADA 2')| 
                               (df_logica['TIPO'] == 'AGENCIA ESPECIALIZADA 3')]
df_logica_zona = df_logica[(df_logica['TIPO'] == 'ZONA/REGION TIPO 1') | 
                           (df_logica['TIPO'] == 'ZONA/REGION ESPECIALIZADA 1')| 
                           (df_logica['TIPO'] == 'ZONA/REGION TIPO 2')| 
                           (df_logica['TIPO'] == 'ZONA/REGION ESPECIALIZADA 3')]

## Creación de todos los productos

## Consolidación real, metas y lógica

In [182]:
df_Consolidado = pd.merge(df_Metas, df_Real,how='left', 
             left_on = ['Producto', 'CodigoAgenciaAnclaje'] ,
             right_on = ['Producto', 'CodigoAgenciaAnclaje'])
print(df_Consolidado.shape)
df_Consolidado["Real"].fillna(0, inplace=True)
df_Consolidado=df_Consolidado.rename(columns = {'TIPO': 'TipoAgencia'})
df_logica_agencia=df_logica_agencia.rename(columns = {'PRODUCTO': 'Producto', 'TIPO': 'TipoAgencia'})
print(df_Consolidado.columns)
print(df_logica_agencia.columns)
# df_logica_agencia = df_logica_agencia[df_logica_agencia["ACTIVO"] == "SI"]
df_Consolidado = pd.merge(df_Consolidado, df_logica_agencia,  how='left', 
             left_on=['Producto', 'TipoAgencia'],
             right_on =['Producto', 'TipoAgencia'] )

(3312, 5)
Index(['Producto', 'CodigoAgenciaAnclaje', 'Metas', 'TipoAgencia', 'Real'], dtype='object')
Index(['CATEGORIA', 'Producto', 'ACTIVO', 'TECHO_PRODUCTO', 'TECHO_P',
       'TECHO_C', 'PORCENTAJE', 'INVERTIBILIDAD', 'TipoAgencia', 'VALOR',
       'ERROR', 'ANIO', 'MES'],
      dtype='object')


C:\Users\mencarna\AppData\Local\Temp\ipykernel_23280\4185556006.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_Consolidado["Real"].fillna(0, inplace=True)


In [183]:
if ajuste == "Si": 
    ajustes = pd.read_excel(ruta,
                   header=[0,1,2],
                   index_col=[0,1],
                   sheet_name='Ajustes')
    ajustes = ajustes.stack(level=1).stack(level=1).stack().unstack(level=2)
    ajustes = ajustes.reset_index()
    ajustes=ajustes.rename(columns = {'level_0':'CodigoAgenciaAnclaje', 'level_1': 'AGENCIA' ,'AGENCIA':'Producto','level_3':'CATEGORIA', 
                                       'METAS': 'Ajuste'})
    ajustes = ajustes[['Producto','CodigoAgenciaAnclaje','Ajuste']]
    df_Consolidado = pd.merge(df_Consolidado, ajustes, how = 'left', 
                           left_on = ['Producto', 'CodigoAgenciaAnclaje'], 
                           right_on = ['Producto', 'CodigoAgenciaAnclaje'])
    df_Consolidado["Ajuste"].fillna(0, inplace=True)
    df_Consolidado['Real'] = df_Consolidado['Real'] + df_Consolidado['Ajuste']
    df_Consolidado.drop(['Ajuste'], axis=1, inplace = True)
    print("Se han aplicado los ajustes, las dimesiones son {}.".format(df_Consolidado.shape))
else: 
    print("No se ejecutaron ajustes.")

Se han aplicado los ajustes, las dimesiones son (3312, 16).


C:\Users\mencarna\AppData\Local\Temp\ipykernel_23280\2740205709.py:6: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  ajustes = ajustes.stack(level=1).stack(level=1).stack().unstack(level=2)
C:\Users\mencarna\AppData\Local\Temp\ipykernel_23280\2740205709.py:6: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  ajustes = ajustes.stack(level=1).stack(level=1).stack().unstack(level=2)
C:\Users\mencarna\AppData\Local\Temp\ipykernel_23280\2740205709.py:14: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace metho

## Región y zona: 

In [184]:
query = f"""
SELECT 
	Agencia, 
	CodigoAgencia AS CodigoAgenciaAnclaje, 
	Region,
	Zona 
FROM [BDDCATALOGOS].[CAT].[CatalogoAgencia]; 
"""
df_agencias = pd.read_sql(query, cnxn)
df_Consolidado = pd.merge(df_Consolidado, df_agencias,  how='left', 
             left_on=['CodigoAgenciaAnclaje'],
             right_on =['CodigoAgenciaAnclaje'] )
print(df_Consolidado.shape)

(3312, 19)


C:\Users\mencarna\AppData\Local\Temp\ipykernel_23280\2529797175.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_agencias = pd.read_sql(query, cnxn)


## Cumplimiento por Zona y Región

In [185]:
df_Real_Region = df_Consolidado[['Producto', 'Real', 'Metas', 'Agencia', 'Region', 'Zona']]
Lista_Servicio  = ['TIEMPO MEDIO DE ESPERA','TIEMPO MEDIO DE ATENCIÓN',
                   'HEAD COUNT OPERATIVO', 'PRECANCELACIONES VOLUNTARIAS SEGUROS', 
                   'TASA ABANDONO', 'INS', 'NIVEL DE ATENCIÓN (SLA)','NPS',
                  'APERTURA DE CUENTA CON AFILIACIÓN BANCA MÓVIL', 'OPERACIÓN', 'MIGRACIÓN','CUMPLIMIENTO CERTIFICACIÓN TRIMESTRAL AGENCIAS']
df_IndicadoresServicio_Zona = df_Real_Region[df_Real_Region['Producto'].isin(Lista_Servicio) ]
df_Real_Region = df_Real_Region[~df_Real_Region['Producto'].isin(Lista_Servicio)]

In [186]:
import os
# Ruta de los archivos SQL
ruta_fstp = '//ECUIO197H02/group_files/Inteligencia de Negocios/IDN-Global/Agencias/QuerysRegionZona/'
# Listar todos los archivos .sql en la ruta
archivos_sql = [archivo for archivo in os.listdir(ruta_fstp) if archivo.endswith('.sql')]
# Leer y concatenar las consultas usando "UNION ALL"
consulta_final = '\nUNION ALL\n'.join([f'{leer_consulta(ruta_fstp + archivo)}' for archivo in archivos_sql])
consulta_final = consulta_final.format(lista = lista, Anio = Anio, Mes = Mes)
df_Indicadores_Servicio_Zona = pd.read_sql(consulta_final, cnxn)
print('La obtención de Datos Reales de la Fuerza de Venta es Exitosa.')

C:\Users\mencarna\AppData\Local\Temp\ipykernel_23280\2599005429.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_Indicadores_Servicio_Zona = pd.read_sql(consulta_final, cnxn)


La obtención de Datos Reales de la Fuerza de Venta es Exitosa.


### Ajuste por lógica del cálculo del NPS: 

In [187]:
df_Real_agrupado = df_Real_Region.groupby(['Producto', 
                                           'Zona',
                                           'Region']).agg({'Real': ['sum'], 
                                                           'Metas': ['sum']})
df_Real_agrupado = df_Real_agrupado.reset_index()
df_Real_agrupado.columns = ['Producto', 'Zona', 'Region', 'Real', 'Metas']

df_IndicadoresServicio_ZonaAgrupado = df_IndicadoresServicio_Zona.groupby(['Producto', 
                                                                           'Zona', 
                                                                           'Region']).agg({'Metas':
                                                                                           ['mean']
                                                                                          })
df_IndicadoresServicio_ZonaAgrupado = df_IndicadoresServicio_ZonaAgrupado.reset_index()
df_IndicadoresServicio_ZonaAgrupado.columns = ['Producto', 'Zona', 'Region', 'Metas']


df_IndicadoresServicio_ZonaAgrupado = pd.merge(df_IndicadoresServicio_ZonaAgrupado, 
                                               df_Indicadores_Servicio_Zona, 
                                               how = 'left', 
                                               left_on = ["Zona", "Producto"], 
                           right_on = ["Zona", "Producto"])

df_Metas_Servicios = pd.read_excel(ruta,sheet_name='Metas Servicio')

df_IndicadoresServicio_ZonaAgrupado = pd.merge(df_IndicadoresServicio_ZonaAgrupado, 
                                                df_Metas_Servicios, 
                                                how = 'left', 
                                                left_on = ["Zona", "Producto"], 
                            right_on = ["NOMBRE ZONA", "PRODUCTO"])

df_IndicadoresServicio_ZonaAgrupado['Metas'] = np.where(
    df_IndicadoresServicio_ZonaAgrupado['META'].notna(), 
    df_IndicadoresServicio_ZonaAgrupado['META'],
    df_IndicadoresServicio_ZonaAgrupado['Metas'])
df_IndicadoresServicio_ZonaAgrupado = df_IndicadoresServicio_ZonaAgrupado[['Producto',
                                                                           'Zona', 'Region', 
                                                                           'Metas','Real']]
df_IndicadoresServicio_ZonaAgrupado["Real"].fillna(0, inplace=True)
df_IndicadoresServicio_ZonaAgrupado = df_IndicadoresServicio_ZonaAgrupado[['Producto', 
                                                                           'Zona',
                                                                           'Region', 'Metas' ,'Real']]

df_Real_agrupado = pd.concat([df_Real_agrupado,df_IndicadoresServicio_ZonaAgrupado])

C:\Users\mencarna\AppData\Local\Temp\ipykernel_23280\371798070.py:38: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_IndicadoresServicio_ZonaAgrupado["Real"].fillna(0, inplace=True)


In [188]:
df_Tipo_Zona = pd.read_excel(ruta,sheet_name='Zona')
df_Tipo_Zona = df_Tipo_Zona.rename(columns = {"NOMBRE ZONA":"Zona"})
df_Tipo_Zona = df_Tipo_Zona[['Zona', 'TIPO']] 
df_Real_agrupado = pd.merge(df_Real_agrupado, df_Tipo_Zona, how = 'left', left_on = ["Zona"], 
                           right_on = ["Zona"])

In [189]:
df_logica_zona = df_logica_zona.rename(columns = {'PRODUCTO': 'Producto'})
df_Consolidado_agrupado = pd.merge(df_Real_agrupado, df_logica_zona,  how='left', 
             left_on=['Producto', 'TIPO'],
             right_on =['Producto', 'TIPO'])

In [190]:
query = f"""
SELECT 
    MAX(FechaAnalisis)
from
			{lista[2]} AS A  WITH (nolock)
		LEFT JOIN 
			[BDDCATALOGOS].[CAT].[CatalogoAgencia] AS B WITH (NOLOCK)
		ON A.CodigoAgencia = B.CodigoAgencia
		where 
			  A.Segmento !='MICROFINANZAS' 
			  and A.FamiliaHomologada not in ('CASAS COMERCIALES','CCSA EMPRESAS','CCSA PERSONAS','MULTILATERALES','PAGOS Y GASTOS')
			  and A.CodigoProducto not in ('267','469','643','947','948','824','194')
			  -- and A.FuerzaVenta IN ('CANAL RED AGENCIAS','CANAL RED MICROFINANZAS')
			  and A.FuerzaVenta IN ('CANAL RED AGENCIAS')
			  and YEAR(A.FechaAnalisis) = {Anio}
			  AND MONTH(A.FechaAnalisis) = {Mes}
			  AND a.UGCE is null
"""

fecha = pd.read_sql(query, cnxn)
df_Consolidado['FECHA_CORTE'] =  pd.to_datetime(fecha.iloc[0,0])
df_Consolidado_agrupado['FECHA_CORTE'] =  pd.to_datetime(fecha.iloc[0,0])

C:\Users\mencarna\AppData\Local\Temp\ipykernel_23280\4051044642.py:20: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  fecha = pd.read_sql(query, cnxn)


## Agrupación por región por Lógica del Cálculo del NPS:

In [191]:
df_Real_Region_fin = df_Consolidado[['Producto', 'Real', 'Metas','Region']]

df_Real_Region_fin_MetasIndicadores = df_Real_Region_fin[df_Real_Region_fin['Producto'].isin(
    Lista_Servicio) ]

df_Real_Region_fin = df_Real_Region_fin[~df_Real_Region_fin['Producto'].isin(Lista_Servicio) ]
df_Real_Region_fin_agrupado = df_Real_Region_fin.groupby(['Producto','Region']).agg({'Real': ['sum'],
                                                                                     'Metas': ['sum']})
df_Real_Region_fin_agrupado = df_Real_Region_fin_agrupado.reset_index()
df_Real_Region_fin_agrupado.columns = ['Producto', 'Region', 'Real', 'Metas']

# Indicadores de servicio por Región:

In [192]:
# Ruta de los archivos SQL
ruta_fstp = '//ECUIO197H02/group_files/Inteligencia de Negocios/IDN-Global/Agencias/QuerysRegion/'
# Listar todos los archivos .sql en la ruta
archivos_sql = [archivo for archivo in os.listdir(ruta_fstp) if archivo.endswith('.sql')]
# Leer y concatenar las consultas usando "UNION ALL"
consulta_final = '\nUNION ALL\n'.join([f'{leer_consulta(ruta_fstp + archivo)}' for archivo in archivos_sql])
consulta_final = consulta_final.format(lista = lista, Anio = Anio, Mes = Mes)
df_Real_Region_fin_agrupado_NPS_Indicadores_region = pd.read_sql(consulta_final, cnxn)

C:\Users\mencarna\AppData\Local\Temp\ipykernel_23280\1427229700.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_Real_Region_fin_agrupado_NPS_Indicadores_region = pd.read_sql(consulta_final, cnxn)


In [193]:
df_Real_Region_fin_MetasIndicadoresAgrupado = df_Real_Region_fin_MetasIndicadores.groupby(
    ['Producto','Region']).agg({'Metas': ['mean']})
df_Real_Region_fin_MetasIndicadoresAgrupado = df_Real_Region_fin_MetasIndicadoresAgrupado.reset_index()
df_Real_Region_fin_MetasIndicadoresAgrupado.columns = ['Producto', 'Region', 'Metas']
# df_Real_Region_fin_MetasIndicadoresAgrupado["Metas"].fillna(0, inplace=True)
df_Real_Region_fin_MetasIndicadoresAgrupado = pd.merge(df_Real_Region_fin_MetasIndicadoresAgrupado,
                                                       df_Real_Region_fin_agrupado_NPS_Indicadores_region,
                                                       how = 'left', left_on = ["Region", "Producto"], 
                                                       right_on = ["Region", "Producto"])
df_Real_Region_fin_MetasIndicadoresAgrupado["Real"].fillna(0, inplace=True)



df_Real_Region_fin_MetasIndicadoresAgrupado = df_Real_Region_fin_MetasIndicadoresAgrupado[['Producto',
                                                                                           'Region',
                                                                                           'Real', 
                                                                                           'Metas']]

df_Real_Region_fin_MetasIndicadoresAgrupado = pd.merge(df_Real_Region_fin_MetasIndicadoresAgrupado, 
                                                df_Metas_Servicios, 
                                                how = 'left', 
                                                left_on = ["Region", "Producto"], 
                            right_on = ["NOMBRE ZONA", "PRODUCTO"])

df_Real_Region_fin_MetasIndicadoresAgrupado['Metas'] = np.where(
    df_Real_Region_fin_MetasIndicadoresAgrupado['META'].notna(), 
    df_Real_Region_fin_MetasIndicadoresAgrupado['META'],
    df_Real_Region_fin_MetasIndicadoresAgrupado['Metas'])
df_Real_Region_fin_MetasIndicadoresAgrupado = df_Real_Region_fin_MetasIndicadoresAgrupado[['Producto', 
                                                                                           'Region', 
                                                                                           'Metas',
                                                                                           'Real']]


C:\Users\mencarna\AppData\Local\Temp\ipykernel_23280\696580524.py:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_Real_Region_fin_MetasIndicadoresAgrupado["Real"].fillna(0, inplace=True)


In [194]:
df_Real_Region_fin_agrupado_NPS_Indicadores_region['Metas'] = 0
df_Real_Region_fin_agrupado_consolidado = pd.concat([df_Real_Region_fin_agrupado,
                                                     df_Real_Region_fin_MetasIndicadoresAgrupado])
df_Real_Region_fin_agrupado_consolidado = df_Real_Region_fin_agrupado_consolidado.dropna(subset = ['Region'])
df_Real_Region_fin_agrupado_consolidado = df_Real_Region_fin_agrupado_consolidado[df_Real_Region_fin_agrupado_consolidado['Region'] != None]

In [195]:
ruta_cat= '//ECUIO197H02/group_files/Inteligencia de Negocios/IDN-Global/Agencias/MetasAgencias'+str(Anio)+str(Mes)+'.xlsx'

df_Tipo_Region = pd.read_excel(ruta_cat,
                   sheet_name='Región')

df_Tipo_Region = df_Tipo_Region.rename(columns = {"NOMBRE REGION":"Region"})

df_Tipo_Region = df_Tipo_Region[['Region', 'TIPO']]

df_Real_Region_fin_agrupado_consolidado = pd.merge(df_Real_Region_fin_agrupado_consolidado,
                                                   df_Tipo_Region, 
                                                   how = 'left', left_on = ["Region"], 
                           right_on = ["Region"])

In [196]:
df_Consolidado_agrupado_Region = pd.merge(df_Real_Region_fin_agrupado_consolidado,
                                          df_logica_zona,  how='left', 
             left_on=['Producto', 'TIPO'],
             right_on =['Producto', 'TIPO'])
revisar = df_Consolidado_agrupado_Region[df_Consolidado_agrupado_Region["INVERTIBILIDAD"].isna()]
revisar["Region"].unique()

array([], dtype=object)

In [197]:
df_Consolidado_agrupado_RegionRelacional = df_Real_Region_fin_agrupado_consolidado[~(df_Real_Region_fin_agrupado_consolidado['Producto'].isin(Lista_Servicio))] 
df_Consolidado_agrupado_RegionRelacional = df_Consolidado_agrupado_RegionRelacional[['Producto', 'Real', 'Metas']]
df_NPS_Relacional = df_Real_Region_fin_agrupado_consolidado[df_Real_Region_fin_agrupado_consolidado['Producto'].isin(Lista_Servicio)]
df_NPS_Relacional = df_NPS_Relacional.groupby(['Producto']).mean(['Real', 'Metas']).reset_index()
df_NPS_Relacional['Region'] = 'RELACIONAL'
#########################################333
df_Real_Region_fin_agrupado_NPS_Indicadores_region_aux=df_Real_Region_fin_agrupado_NPS_Indicadores_region.rename(columns = {"Real":"REAL", "Metas":"METAS"})
df_NPS_Relacional = pd.merge(df_NPS_Relacional, 
                                                df_Real_Region_fin_agrupado_NPS_Indicadores_region_aux, 
                                                how = 'left', 
                                                left_on = ["Region", "Producto"], 
                            right_on = ["Region", "Producto"])

df_NPS_Relacional['Real'] = np.where(
    df_NPS_Relacional['REAL'].notna(), 
    df_NPS_Relacional['REAL'],
    df_NPS_Relacional['Real'])
df_NPS_Relacional = pd.merge(df_NPS_Relacional, 
                                                df_Metas_Servicios, 
                                                how = 'left', 
                                                left_on = ["Region", "Producto"], 
                            right_on = ["NOMBRE ZONA", "PRODUCTO"])

df_NPS_Relacional['Metas'] = np.where(
    df_NPS_Relacional['META'].notna(), 
    df_NPS_Relacional['META'],
    df_NPS_Relacional['Metas'])
df_NPS_Relacional=df_NPS_Relacional[['Producto', 'Real', 'Metas','Region']]
#####################################33
df_Consolidado_agrupado_RegionRelacional = df_Consolidado_agrupado_RegionRelacional.groupby(['Producto']).sum(['Real', 'Metas']).reset_index()
df_Consolidado_agrupado_RegionRelacional['Region'] = 'RELACIONAL'
## df_Consolidado_agrupado_RegionRelacional
df_Consolidado_agrupado_RegionRelacional = pd.concat([df_Consolidado_agrupado_RegionRelacional, df_NPS_Relacional])
df_Consolidado_agrupado_RegionRelacional['TIPO'] = 'ZONA/REGION TIPO 1'

df_Consolidado_agrupado_RegionRelacional = pd.merge(df_Consolidado_agrupado_RegionRelacional, df_logica_zona,  how='left', 
             left_on=['Producto', 'TIPO'],
             right_on =['Producto', 'TIPO'])

### Ajustes de estructura por nueva lógica - aqui

In [198]:
df_Consolidado = df_Consolidado.rename(columns = {'TipoAgencia': 'TIPO_AGENCIA'})
columnas = df_Consolidado.columns.tolist()
columnas.append(columnas.pop(columnas.index('TIPO_AGENCIA')))
df_Consolidado = df_Consolidado[columnas]

### Ajuste de estructura por nueva lógica:

In [199]:
columnas = df_Consolidado_agrupado.columns.tolist()
columnas.append(columnas.pop(columnas.index('TIPO')))
df_Consolidado_agrupado = df_Consolidado_agrupado[columnas]

### Ajuste de estructura por nueva lógica Región:

In [200]:
df_Consolidado_agrupado_RegionRelacional['FECHA_CORTE'] =  pd.to_datetime(fecha.iloc[0,0])
columnas = df_Consolidado_agrupado_RegionRelacional.columns.tolist()
columnas.append(columnas.pop(columnas.index('TIPO')))
df_Consolidado_agrupado_RegionRelacional = df_Consolidado_agrupado_RegionRelacional[columnas]

df_Consolidado_agrupado_Region['FECHA_CORTE'] =  pd.to_datetime(fecha.iloc[0,0])
columnas = df_Consolidado_agrupado_Region.columns.tolist()
columnas.append(columnas.pop(columnas.index('TIPO')))
df_Consolidado_agrupado_Region = df_Consolidado_agrupado_Region[columnas]

In [201]:
df_Consolidado_agrupado_Region = pd.concat([df_Consolidado_agrupado_Region,df_Consolidado_agrupado_RegionRelacional])

## Creación y actualización de información en las tablas de SQL

In [202]:
cursor.execute("TRUNCATE TABLE [DATALAB_IDN].[IDN].[Seguimiento_Agencias_Nivel_Uno]")
for index, row in df_Consolidado.iterrows():
    cursor.execute("""INSERT INTO [DATALAB_IDN].[IDN].[Seguimiento_Agencias_Nivel_Uno]
    (Producto, CodigoAgenciaAnclaje, Metas, Real, CATEGORIA, ACTIVO, TECHO_PRODUCTO, 
    TECHO_P, TECHO_C, PORCENTAJE, INVERTIBILIDAD, VALOR, ERROR, 
    ANIO, MES, Agencia, Region, Zona, FECHA_CORTE, TIPO_AGENCIA) 
    values(?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)""",
                   row.Producto,row.CodigoAgenciaAnclaje,row.Metas,row.Real,row.CATEGORIA,row.ACTIVO,row.TECHO_PRODUCTO,
                   row.TECHO_P,row.TECHO_C,row.PORCENTAJE,row.INVERTIBILIDAD,row.VALOR,row.ERROR,
                   row.ANIO,row.MES, row.Agencia, row.Region,row.Zona,row.FECHA_CORTE,row.TIPO_AGENCIA)
    cnxn.commit()
print('Completado')

Completado


In [203]:
cursor.execute(f"""
	DELETE FROM [DATALAB_IDN].[HIS].[Seguimiento_Agencias_Nivel_Uno]
	WHERE YEAR(FECHA_CORTE) = {Anio} AND MONTH(FECHA_CORTE) = {Mes}
	INSERT INTO [DATALAB_IDN].[HIS].[Seguimiento_Agencias_Nivel_Uno]
	SELECT * FROM [DATALAB_IDN].[IDN].[Seguimiento_Agencias_Nivel_Uno]
""")
cnxn.commit()
print('Fin Proceso Agencias.')

Fin Proceso Agencias.


In [204]:
df_Consolidado[df_Consolidado.isnull().any(axis=1)]


,Producto,CodigoAgenciaAnclaje,Metas,Real,CATEGORIA,ACTIVO,TECHO_PRODUCTO,TECHO_P,TECHO_C,PORCENTAJE,INVERTIBILIDAD,VALOR,ERROR,ANIO,MES,Agencia,Region,Zona,FECHA_CORTE,TIPO_AGENCIA


## Seguimiento por zona

In [205]:
cursor.execute("TRUNCATE TABLE [DATALAB_IDN].[IDN].[Seguimiento_Agencias_Nivel_Dos]")
for index, row in df_Consolidado_agrupado.iterrows():
    cursor.execute("""INSERT INTO [DATALAB_IDN].[IDN].[Seguimiento_Agencias_Nivel_Dos]
    (Producto, Zona, Region, Real, Metas, CATEGORIA, ACTIVO, 
    TECHO_PRODUCTO, TECHO_P, TECHO_C, PORCENTAJE, INVERTIBILIDAD, 
    VALOR, ERROR, ANIO, MES, FECHA_CORTE, TIPO) 
    values(?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)""",
                   row.Producto,row.Zona,row.Region,row.Real, row.Metas , row.CATEGORIA,row.ACTIVO,
                   row.TECHO_PRODUCTO, row.TECHO_P,row.TECHO_C,row.PORCENTAJE,row.INVERTIBILIDAD,
                   row.VALOR,row.ERROR, row.ANIO,row.MES, row.FECHA_CORTE, row.TIPO)
    cnxn.commit()
print('Completado')

Completado


In [206]:
cursor.execute(f"""
	DELETE FROM [DATALAB_IDN].[HIS].[Seguimiento_Agencias_Nivel_Dos]
	WHERE YEAR(FECHA_CORTE) = {Anio} AND MONTH(FECHA_CORTE) = {Mes}
	INSERT INTO [DATALAB_IDN].[HIS].[Seguimiento_Agencias_Nivel_Dos]
	SELECT * FROM [DATALAB_IDN].[IDN].[Seguimiento_Agencias_Nivel_Dos]
""")
cnxn.commit()
print('Fin Proceso Zona.')

Fin Proceso Zona.


## Seguimiento por Región:

In [207]:
cursor.execute("TRUNCATE TABLE [DATALAB_IDN].[IDN].[Seguimiento_Agencias_Nivel_Tres]")
for index, row in df_Consolidado_agrupado_Region.iterrows():
    cursor.execute("""INSERT INTO [DATALAB_IDN].[IDN].[Seguimiento_Agencias_Nivel_Tres]
    (Producto, Region, Real, Metas, CATEGORIA, ACTIVO, 
    TECHO_PRODUCTO, TECHO_P, TECHO_C, PORCENTAJE, INVERTIBILIDAD, 
    VALOR, ERROR, ANIO, MES, FECHA_CORTE, TIPO) 
    values(?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)""",
                   row.Producto,row.Region,row.Real, row.Metas , row.CATEGORIA,row.ACTIVO,
                   row.TECHO_PRODUCTO, row.TECHO_P,row.TECHO_C,row.PORCENTAJE,row.INVERTIBILIDAD,
                   row.VALOR,row.ERROR, row.ANIO,row.MES, row.FECHA_CORTE, row.TIPO)
    cnxn.commit()
print('Completado')

Completado


In [208]:
cursor.execute(f"""
	DELETE FROM [DATALAB_IDN].[HIS].[Seguimiento_Agencias_Nivel_Tres]
	WHERE YEAR(FECHA_CORTE) = {Anio} AND MONTH(FECHA_CORTE) = {Mes}
	INSERT INTO [DATALAB_IDN].[HIS].[Seguimiento_Agencias_Nivel_Tres]
	SELECT * FROM [DATALAB_IDN].[IDN].[Seguimiento_Agencias_Nivel_Tres]
""")
cnxn.commit()
print('Fin Proceso Región.')

Fin Proceso Región.


# Agencias Compartidas

In [209]:
df_AgenciasCompartidas = pd.read_excel(ruta, sheet_name='Agencia Compartida')
df_AgenciasCompartidas = df_AgenciasCompartidas.rename(columns = {"COD":"CodigoAgenciaAnclaje", "TIPO":"TipoAgencia", "MEDICION":"medicion"})
df_AgenciasComponentes = df_AgenciasCompartidas.groupby(['medicion']).agg({'CodigoAgenciaAnclaje':["min", "max"]})
df_AgenciasComponentes.columns = ['Agenciamin', 'Agenciamax']
df_AgenciasComponentes = df_AgenciasComponentes.reset_index()

In [210]:
# df_AgenciasCompartidas
df_RealCompartida = df_Consolidado[['CodigoAgenciaAnclaje', 'Real', 'Producto']]

df_ConsolidadoAgenciasCompartidas = df_AgenciasCompartidas.merge(df_RealCompartida, on = ["CodigoAgenciaAnclaje"], how = "inner")
df_ConsolidadoAgenciasCompartidasParte1 = df_ConsolidadoAgenciasCompartidas[~df_ConsolidadoAgenciasCompartidas['Producto'].isin(Lista_Servicio)].groupby(['medicion', 'Producto']).agg({"Real":["sum"]})
df_ConsolidadoAgenciasCompartidasParte2 = df_ConsolidadoAgenciasCompartidas[df_ConsolidadoAgenciasCompartidas['Producto'].isin(Lista_Servicio)].groupby(['medicion', 'Producto']).agg({"Real":["mean"]})
df_ConsolidadoAgenciasCompartidasParte1 = df_ConsolidadoAgenciasCompartidasParte1.reset_index()
df_ConsolidadoAgenciasCompartidasParte2 = df_ConsolidadoAgenciasCompartidasParte2.reset_index()
df_ConsolidadoAgenciasCompartidasParte1.columns = df_ConsolidadoAgenciasCompartidasParte1.columns.droplevel(1)
df_ConsolidadoAgenciasCompartidasParte2.columns = df_ConsolidadoAgenciasCompartidasParte2.columns.droplevel(1)
df_ConsolidadoAgenciasCompartidasFin = pd.concat([df_ConsolidadoAgenciasCompartidasParte1, df_ConsolidadoAgenciasCompartidasParte2])
df_ConsolidadoAgenciasCompartidasFin = df_ConsolidadoAgenciasCompartidasFin.rename(columns = {'medicion':'CodigoAgenciaAnclaje'})
df_ConsolidadoAgenciasCompartidasFin = df_ConsolidadoAgenciasCompartidasFin[['Producto', 'CodigoAgenciaAnclaje', 'Real']]
# Consolidacion de reales: 
df_ConsolidadoAgenciasCompartidasFin.shape

(198, 3)

## Consolidación de Metas:

In [211]:
df_ConsolidadoAgenciasCompartidasMetas = df_AgenciasCompartidas.merge(df_Metas, on = ["CodigoAgenciaAnclaje"], how = "inner")
df_ConsolidadoAgenciasCompartidasMetasParte1 = df_ConsolidadoAgenciasCompartidasMetas[~df_ConsolidadoAgenciasCompartidasMetas['Producto'].isin(Lista_Servicio)].groupby(['medicion', 'Producto', 'TipoAgencia']).agg({"Metas":["sum"]})
df_ConsolidadoAgenciasCompartidasMetasParte2 = df_ConsolidadoAgenciasCompartidasMetas[df_ConsolidadoAgenciasCompartidasMetas['Producto'].isin(Lista_Servicio)].groupby(['medicion', 'Producto', 'TipoAgencia']).agg({"Metas":["mean"]})
df_ConsolidadoAgenciasCompartidasMetasParte1 = df_ConsolidadoAgenciasCompartidasMetasParte1.reset_index()
df_ConsolidadoAgenciasCompartidasMetasParte2 = df_ConsolidadoAgenciasCompartidasMetasParte2.reset_index()
df_ConsolidadoAgenciasCompartidasMetasParte1.columns = df_ConsolidadoAgenciasCompartidasMetasParte1.columns.droplevel(1)
df_ConsolidadoAgenciasCompartidasMetasParte2.columns = df_ConsolidadoAgenciasCompartidasMetasParte2.columns.droplevel(1)
df_ConsolidadoAgenciasCompartidasMetasFin = pd.concat([df_ConsolidadoAgenciasCompartidasMetasParte1, df_ConsolidadoAgenciasCompartidasMetasParte2])
df_ConsolidadoAgenciasCompartidasMetasFin = df_ConsolidadoAgenciasCompartidasMetasFin.rename(columns = {'medicion':'CodigoAgenciaAnclaje'})
df_ConsolidadoAgenciasCompartidasMetasFin = df_ConsolidadoAgenciasCompartidasMetasFin[['Producto', 'CodigoAgenciaAnclaje', 'Metas', 'TipoAgencia']]
# Consolidacion de reales: 
df_ConsolidadoAgenciasCompartidasMetasFin.shape

(198, 4)

## Consolidación real, metas y lógica:

In [212]:
df_ConsolidadoAgenciasCompartidas = pd.merge(df_ConsolidadoAgenciasCompartidasMetasFin, df_ConsolidadoAgenciasCompartidasFin,how='left', 
             left_on = ['Producto', 'CodigoAgenciaAnclaje'] ,
             right_on = ['Producto', 'CodigoAgenciaAnclaje'])
print(df_ConsolidadoAgenciasCompartidas.shape)
df_ConsolidadoAgenciasCompartidas["Real"].fillna(0, inplace=True)

(198, 5)


C:\Users\mencarna\AppData\Local\Temp\ipykernel_23280\3085566596.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_ConsolidadoAgenciasCompartidas["Real"].fillna(0, inplace=True)


In [213]:
df_ConsolidadoAgenciasCompartidas = pd.merge(df_ConsolidadoAgenciasCompartidas, df_logica_zona,  how='left', 
             left_on=['Producto', 'TipoAgencia'],
             right_on =['Producto', 'TIPO'] )
print(df_ConsolidadoAgenciasCompartidas.shape)
df_ConsolidadoAgenciasCompartidas = df_ConsolidadoAgenciasCompartidas.drop('TIPO', axis = 1)
df_ConsolidadoAgenciasCompartidas = df_ConsolidadoAgenciasCompartidas.rename(columns = {"CodigoAgenciaAnclaje":"CodigoAgenciaCompartida"})
#df_ConsolidadoAgenciasCompartidas
df_ConsolidadoAgenciasCompartidas['FECHA_CORTE'] =  pd.to_datetime(fecha.iloc[0,0])
df_ConsolidadoAgenciasCompartidas['Agencia'] = 'Grupo ' + df_ConsolidadoAgenciasCompartidas['CodigoAgenciaCompartida'].astype(str)

(198, 17)


In [214]:
df_AgenciasComponentes = df_AgenciasComponentes.rename(columns = {"medicion":"CodigoAgenciaCompartida"})
df_ConsolidadoAgenciasCompartidas = pd.merge(df_ConsolidadoAgenciasCompartidas, df_AgenciasComponentes, how = 'left', 
                                            left_on=['CodigoAgenciaCompartida'],
                                            right_on=['CodigoAgenciaCompartida']
                                            )

In [215]:
print(df_logica_zona['TIPO'].unique())

a=df_ConsolidadoAgenciasCompartidas[df_ConsolidadoAgenciasCompartidas['CATEGORIA'].isna()]
a['Producto'].unique()

['ZONA/REGION TIPO 1' 'ZONA/REGION ESPECIALIZADA 1' 'ZONA/REGION TIPO 2']


array([], dtype=object)

In [216]:
cursor.execute("TRUNCATE TABLE [DATALAB_IDN].[IDN].[Seguimiento_Agencias_Nivel_Uno_Agrupado]")
for index, row in df_ConsolidadoAgenciasCompartidas.iterrows():
    cursor.execute("""INSERT INTO [DATALAB_IDN].[IDN].[Seguimiento_Agencias_Nivel_Uno_Agrupado]
    (Producto, CodigoAgenciaCompartida, Real, Metas, CATEGORIA, ACTIVO, 
    TECHO_PRODUCTO, TECHO_P, TECHO_C, PORCENTAJE, INVERTIBILIDAD, 
    VALOR, ERROR, ANIO, MES, FECHA_CORTE, TIPO, Agencia, AgenciaUno, AgenciaDos) 
    values(?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)""",
                   row.Producto,row.CodigoAgenciaCompartida,row.Real, row.Metas , row.CATEGORIA,row.ACTIVO,
                   row.TECHO_PRODUCTO, row.TECHO_P,row.TECHO_C,row.PORCENTAJE,row.INVERTIBILIDAD,
                   row.VALOR,row.ERROR, row.ANIO,row.MES, row.FECHA_CORTE, row.TipoAgencia, row.Agencia, row.Agenciamin, row.Agenciamax)
    cnxn.commit()
print('Completado')

cursor.execute(f"""
	DELETE FROM [DATALAB_IDN].[HIS].[Seguimiento_Agencias_Nivel_Uno_Agrupado]
	WHERE YEAR(FECHA_CORTE) = {Anio} AND MONTH(FECHA_CORTE) = {Mes}
	INSERT INTO [DATALAB_IDN].[HIS].[Seguimiento_Agencias_Nivel_Uno_Agrupado]
	SELECT * FROM [DATALAB_IDN].[IDN].[Seguimiento_Agencias_Nivel_Uno_Agrupado]
""")
cnxn.commit()
print('Fin Proceso Agencia Compartida.')

Completado
Fin Proceso Agencia Compartida.


In [217]:
catalogo = df_AgenciasCompartidas.rename(columns = {'medicion':'CodigoAgenciaCompartida'})
catalogo.dtypes
cursor.execute("TRUNCATE TABLE [DATALAB_IDN].[IDN].[Catalogo_Agencias_Agrupadas]")
for index, row in catalogo.iterrows():
    cursor.execute("""INSERT INTO [DATALAB_IDN].[IDN].[Catalogo_Agencias_Agrupadas]
    (CodigoAgenciaAnclaje, CodigoAgenciaCompartida) 
    values(?,?)""",
                   row.CodigoAgenciaAnclaje,row.CodigoAgenciaCompartida)
    cnxn.commit()
print('Completado')

Completado


In [218]:
queryReguladores = f"""
DELETE [DATALAB_IDN].[IDN].[REGULADORES_SEGUROS_AGENCIA_COMPARTIDA] WHERE ANIO = {Anio} AND MES = {Mes};
  
INSERT INTO [DATALAB_IDN].[IDN].[REGULADORES_SEGUROS_AGENCIA_COMPARTIDA]
SELECT
	DT.ANIO, 
	DT.MES, 
	DT.CodigoAgenciaCompartida, 
	AVG([REAL]) AS [REAL],
	CONCAT(DT.CodigoAgenciaCompartida, DT.ANIO, DT.MES) AS CLAVE 
-- INTO [DATALAB_IDN].[IDN].[REGULADORES_SEGUROS_AGENCIA_COMPARTIDA]
FROM (
	SELECT 
		A.*, 
		B.CodigoAgenciaCompartida
	FROM [DATALAB_IDN].[IDN].[REGULADORES_SEGUROS_AGENCIA] AS A
	INNER JOIN [DATALAB_IDN].[IDN].[Catalogo_Agencias_Agrupadas] AS B
	ON A.COD_AGENCIA = B.CodigoAgenciaAnclaje
	WHERE A.ANIO = {Anio} AND A.MES = {Mes}
) DT
GROUP BY 
	DT.ANIO, 
	DT.MES, 
	DT.CodigoAgenciaCompartida,
	CONCAT(DT.CodigoAgenciaCompartida, DT.ANIO, DT.MES);
 
-- REGULADORES DE SEGUROS PARA CONTROL:
 
DELETE [DATALAB_IDN].[IDN].[REGULADORES_AGENCIA_COMPARTIDA] WHERE ANIO = {Anio} AND MES = {Mes};
 
INSERT INTO [DATALAB_IDN].[IDN].[REGULADORES_AGENCIA_COMPARTIDA]
SELECT
	DT.ANIO, 
	DT.MES, 
	DT.CodigoAgenciaCompartida, 
	AVG([REAL]) AS [REAL],
	CONCAT(DT.CodigoAgenciaCompartida, DT.ANIO, DT.MES) AS CLAVE 
-- INTO [DATALAB_IDN].[IDN].[REGULADORES_AGENCIA_COMPARTIDA]
FROM (
	SELECT 
		A.*, 
		B.CodigoAgenciaCompartida
	FROM [DATALAB_IDN].[IDN].[REGULADORES_AGENCIA] AS A
	INNER JOIN [DATALAB_IDN].[IDN].[Catalogo_Agencias_Agrupadas] AS B
	ON A.COD_AGENCIA = B.CodigoAgenciaAnclaje
	WHERE A.ANIO = {Anio} AND A.MES =  {Mes}
	UNION ALL 
	select 
	d.anio AS ANIO,
	d.mes AS MES,
	d.CodigoAgenciaAnclaje AS COD_AGENCIA,
	iif(d.NumeroCuentas=0,0,cast(d.Considerados as float)/cast(d.NumeroCuentas as float )) REAL,
	d.CodigoAgenciaCompartida
	from (
	select 
	c.CodigoAgenciaAnclaje,
	c.Total as NumeroCuentas,
	c.Considerados,
	c.anio,
	c.mes,
	B.CodigoAgenciaCompartida
	from (
		select 
		c.CodigoAgenciaAnclaje,
		a.anio,
		a.mes,
		count(distinct(CASE WHEN a.nombreProductoCuentasCrm is not null and a.observacionValidacion not in ('El cliente ya cuenta con afiliacion App','Cuenta no consta en base de datos') and d.Edad>=18 and d.Edad<=70 THEN a.identificacion END)) as Total,
		count(distinct(CASE WHEN a.observacionValidacion in  ('Considerado') and a.nombreProductoCuentasCrm is not null and d.Edad>=18 and d.Edad<=70 then a.identificacion end)) as Considerados		
		from BDDELIVERY.dbo.tmp_bss_view_seguimiento_afiliaciones_aperturas as  a with(nolock)
		left join [DATALAB_MISNEGOCIOS].[HIS].[AsesoresAgencias] as b on (a.usuarioOportunidad=b.Usuario)
		and (a.anio=b.Anio) and (a.mes=b.Mes)
		LEFT JOIN [BDDCATALOGOS].[CAT].[CatalogoAgencia] AS c WITH (NOLOCK)	
		ON b.CodigoAgencia = c.CodigoAgencia
		LEFT JOIN [BDDELIVERY].[INFNEG].[SegmentacionMensual] d
		ON a.identificacion=d.Identificacion
		where b.TipoAsesorProductividad in ('MULTIFUNCION') 
		and observacionValidacion <> 'El numeroOperacion corresponde a la identificacion' 
		and a.anio={Anio} and a.mes={Mes}
		group by 
		c.CodigoAgenciaAnclaje, A.anio, A.mes
	) as c 
	INNER JOIN [DATALAB_IDN].[IDN].[Catalogo_Agencias_Agrupadas] AS B
	ON C.CodigoAgenciaAnclaje = B.CodigoAgenciaAnclaje
	) as d
	WHERE ANIO = {Anio} AND MES = {Mes}
) DT
GROUP BY 
	DT.ANIO, 
	DT.MES, 
	DT.CodigoAgenciaCompartida,
	CONCAT(DT.CodigoAgenciaCompartida, DT.ANIO, DT.MES);"""

cursor.execute(queryReguladores)
cnxn.commit()

## Ajuste INS

In [219]:
queryINS = f"""
-----------------------VIRTUAL---------------------------
--UIO--
--AGENCIA--
UPDATE a
set a.Real=93.67
from
[DATALAB_IDN].[HIS].[Seguimiento_Agencias_Nivel_Uno] a
where a.anio={Anio} and a.mes={Mes} and producto='INS' AND CODIGOAGENCIAANCLAJE=338
--ZONA--
UPDATE a
set a.Real=(SELECT Real from [DATALAB_IDN].[HIS].[Seguimiento_Agencias_Nivel_Uno] a where a.anio={Anio} and a.mes={Mes} and producto='INS' AND CODIGOAGENCIAANCLAJE=338),
	a.TECHO_P=(SELECT TECHO_P from [DATALAB_IDN].[HIS].[Seguimiento_Agencias_Nivel_Uno] a where a.anio={Anio} and a.mes={Mes} and producto='INS' AND CODIGOAGENCIAANCLAJE=338),
	A.TECHO_C=(SELECT TECHO_C from [DATALAB_IDN].[HIS].[Seguimiento_Agencias_Nivel_Uno] a where a.anio={Anio} and a.mes={Mes} and producto='INS' AND CODIGOAGENCIAANCLAJE=338)
from
[DATALAB_IDN].[HIS].[Seguimiento_Agencias_Nivel_Dos] a
where a.anio={Anio} and a.mes={Mes} and producto='INS' AND ZONA='ZONA VIRTUAL UIO'
--GYE--
--AGENCIA--
UPDATE a 
set a.Real=100
from 
[DATALAB_IDN].[HIS].[Seguimiento_Agencias_Nivel_Uno] a
where a.anio={Anio} and a.mes={Mes} and producto='INS' AND CODIGOAGENCIAANCLAJE=339
--ZONA--
UPDATE a
set a.Real=(SELECT Real from [DATALAB_IDN].[HIS].[Seguimiento_Agencias_Nivel_Uno] a where a.anio={Anio} and a.mes={Mes} and producto='INS' AND CODIGOAGENCIAANCLAJE=339),
	a.TECHO_P=(SELECT TECHO_P from [DATALAB_IDN].[HIS].[Seguimiento_Agencias_Nivel_Uno] a where a.anio={Anio} and a.mes={Mes} and producto='INS' AND CODIGOAGENCIAANCLAJE=339),
	A.TECHO_C=(SELECT TECHO_C from [DATALAB_IDN].[HIS].[Seguimiento_Agencias_Nivel_Uno] a where a.anio={Anio} and a.mes={Mes} and producto='INS' AND CODIGOAGENCIAANCLAJE=339)
from
[DATALAB_IDN].[HIS].[Seguimiento_Agencias_Nivel_Dos] a
where a.anio={Anio} and a.mes={Mes} and producto='INS' AND ZONA='ZONA COSTA 8 VIRTUAL GYE'
---------------------------------------------------------
UPDATE  A 
SET TECHO_P=B.Piso,
	TECHO_C=B.Techo
FROM [DATALAB_IDN].[HIS].[Seguimiento_Agencias_Nivel_Uno] a
inner JOIN (select codigoAgencia, CAST(REPLACE(piso, ',', '.') AS DECIMAL(10,2)) as Piso, CAST(REPLACE(Techo, ',', '.') AS DECIMAL(10,2)) as Techo  
from [DATALAB_IDN].[dbo].[metasIndicadoresAgencias2] where anio={Anio} and mes={Mes} and tipo='INS' ) b on a.codigoagenciaanclaje=b.codigoAgencia
where a.anio={Anio} and a.mes={Mes} and a.producto='INS';
UPDATE  A 
SET TECHO_P=B.Piso,
	TECHO_C=B.Techo
FROM [DATALAB_IDN].[HIS].[Seguimiento_Agencias_Nivel_Dos] a
INNER JOIN (select UPPER(region) region, CAST(REPLACE(piso, ',', '.') AS DECIMAL(10,2)) as Piso, CAST(REPLACE(Techo, ',', '.') AS DECIMAL(10,2)) as Techo  
from [DATALAB_IDN].[dbo].[metasIndicadoresZonaTemp] where anio={Anio} and mes={Mes} and tipo='INS' ) b on a.zona=b.region
where a.anio={Anio} and a.mes={Mes} and a.producto='INS';
UPDATE  A 
SET TECHO_P=B.Piso,
	TECHO_C=B.Techo
FROM [DATALAB_IDN].[HIS].[Seguimiento_Agencias_Nivel_Tres] a
INNER JOIN (SELECT 'RELACIONAL' Region,sum(meta) as meta, sum(piso) as piso, sum(techo) techo FROM [DATALAB_IDN].[dbo].[metasIndicadoresGlobalTemp]
where anio={Anio} and mes={Mes} and tipo ='INS'
UNION ALL
SELECT Region, Meta, Piso, Techo FROM [DATALAB_IDN].[dbo].[metasIndicadoresRegionTemp]
WHERE ANIO={Anio} AND MES={Mes} and tipo='INS') B ON A.REGION=B.REGION
where a.anio={Anio} and a.mes={Mes} and a.producto='INS'
UPDATE A
SET Real=b.ins
from
[DATALAB_IDN].[HIS].[Seguimiento_Agencias_Nivel_Uno_Agrupado] a
INNER JOIN (
select 
b.codigoagenciaCompartida,
avg(iif(a.Real<=techo_c and a.Real>=techo_p,1,iif(a.Real>techo_c,a.real/techo_c,a.real/techo_p))) as INS
from [DATALAB_IDN].[HIS].[Seguimiento_Agencias_Nivel_Uno] a
inner join [DATALAB_IDN].[IDN].[Catalogo_Agencias_Agrupadas] AS B
ON A.CodigoAgenciaAnclaje = B.CodigoAgenciaAnclaje
where a.anio={Anio} and a.mes={Mes} and a.producto='INS'
group by b.codigoagenciaCompartida
) b ON A.CodigoAgenciaCompartida=b.codigoAgenciaCompartida
where a.anio={Anio} and a.mes={Mes} and producto='INS'
update a 
set metas=1
from
[DATALAB_IDN].[HIS].[Seguimiento_Agencias_Nivel_Uno_Agrupado] a
where a.anio={Anio} and a.mes={Mes} and a.producto='INS'
"""
cursor.execute(queryINS)
cnxn.commit()

## Usuarios

In [220]:
if Act_usuarios == 'Si':
    import pandas as pd
    ruta_fstp = '//ECUIO197H02/group_files/Inteligencia de Negocios/IDN-Global/Agencias/Usuarios/'
    ruta = ruta_fstp+'Listado_usuarios.xlsx'
    df_Us_Region = pd.read_excel(ruta,
                    header=[0],
                    sheet_name='Regiones')
    df_Us_Agencias = pd.read_excel(ruta,
                    header=[0],
                    sheet_name='Agencias')
    df_Us_Multiagencias = pd.read_excel(ruta,
                    header=[0],
                    sheet_name='Multiagencias')
    df_Us_Zonas = pd.read_excel(ruta,
                    header=[0],
                    sheet_name='Zonas')
    cursor.execute("TRUNCATE TABLE [DATALAB_IDN].[IDN].[UsuariosBSCAgencias]")
    for index, row in df_Us_Agencias.iterrows():
        cursor.execute("""INSERT INTO [DATALAB_IDN].[IDN].[UsuariosBSCAgencias]
        (Correo, Agencia) 
        values(?,?)""",
                    row.Correo, row.Agencia)
        cnxn.commit()
    cursor.execute("TRUNCATE TABLE [DATALAB_IDN].[IDN].[UsuariosBSCZonas]")
    for index, row in df_Us_Zonas.iterrows():
        cursor.execute("""INSERT INTO [DATALAB_IDN].[IDN].[UsuariosBSCZonas]
        (Correo, Zona) 
        values(?,?)""",
                    row.correo, row.Zona)
        cnxn.commit()
    cursor.execute("TRUNCATE TABLE [DATALAB_IDN].[IDN].[UsuariosBSCRegiones]")
    for index, row in df_Us_Region.iterrows():
        cursor.execute("""INSERT INTO [DATALAB_IDN].[IDN].[UsuariosBSCRegiones]
        (correo, Region) 
        values(?,?)""",
                    row.Correo, row.Region)
        cnxn.commit()
    cursor.execute("TRUNCATE TABLE [DATALAB_IDN].[IDN].[UsuariosBSCMultiagencias]")
    for index, row in df_Us_Multiagencias.iterrows():
        cursor.execute("""INSERT INTO [DATALAB_IDN].[IDN].[UsuariosBSCMultiagencias]
        (Correo, Multiagencia) 
        values(?,?)""",
                    row.Correo, row.Multiagencia)
        cnxn.commit()
    print('Completado')